In [2]:
import dspy

In [3]:
lm = dspy.LM("anaconda/openhermes-2.5-mistral-7b/q4_k_m", cache=False)
dspy.configure(lm=lm)

In [4]:
fact_checking = dspy.asyncify(dspy.ChainOfThought("claims -> verdicts: list[bool]"))

res = await fact_checking(
    claims=["Python was released in 1991.", "Python is a compiled language."]
)
res

Prediction(
    reasoning='Verdict for claim 1: False, as Python was released in 1991.\nVerdict for claim 2: False, as Python is an interpreted language, not a compiled one.',
    verdicts=[False, False]
)